In [1]:
import pandas as pd
from influxdb import DataFrameClient

In [2]:
user = 'root'
password = 'root'
dbname = 'base47'
host='localhost'
port=32768
# Temporarily avoid line protocol time conversion issues #412, #426, #431.
protocol = 'json'
client = DataFrameClient(host, port, user, password, dbname)

In [3]:
print("Create pandas DataFrame")
df = pd.DataFrame(data=list(range(30)),
                  index=pd.date_range(start='2017-11-16',
                                      periods=30, freq='H'))

Create pandas DataFrame


In [23]:
gas=pd.read_csv('data/gas_ft.csv',parse_dates=True,index_col='ts').drop('measurement_unit',axis=1)

In [25]:
#client.create_database(dbname)
client.query("show databases")

ResultSet({'(u'databases', None)': [{u'name': u'_internal'}, {u'name': u'test'}, {u'name': u'base'}, {u'name': u'base2'}, {u'name': u'base47'}]})

In [44]:
#for i in range(len(gas)/500):
n=5000
for i in range(5):
    print(i),
    print("Writing batch "+str(i)+" of "+str(n)+" elements to Influx | progress: "+str(round(i*100.0/(len(gas)/n),2)))+"%"
    client.write_points(gas[n*i:n*(i+1)], 'gas', protocol=protocol)

0 Writing batch 0 of 5000 elements to Influx | progress: 0.0%
1 Writing batch 1 of 5000 elements to Influx | progress: 1.14%
2 Writing batch 2 of 5000 elements to Influx | progress: 2.27%
3 Writing batch 3 of 5000 elements to Influx | progress: 3.41%
4 Writing batch 4 of 5000 elements to Influx | progress: 4.55%


In [ ]:
#https://influxdb-python.readthedocs.io/en/latest/api-documentation.html#dataframeclient

In [45]:
client.write_points(gas, 'gas', protocol=protocol, batch_size=5000)

True

In [47]:
client.write_points(gas, 'gas2', protocol=protocol, tag_columns=['monitor_id'], field_columns=['measurement'], batch_size=5000)

True

In [13]:

print("Write DataFrame with Tags")
client.write_points(df, 'demo',
                    {'k1': 'v1', 'k2': 'v2'}, protocol=protocol)

Write DataFrame with Tags


True

In [14]:
print("Read DataFrame")
client.query("select * from demo")

Read DataFrame


defaultdict(list, {u'demo':                              k1    k2  val
             2017-11-16 00:00:00+00:00  None  None    0
             2017-11-16 00:00:00+00:00    v1    v2    0
             2017-11-16 01:00:00+00:00  None  None    1
             2017-11-16 01:00:00+00:00    v1    v2    1
             2017-11-16 02:00:00+00:00  None  None    2
             2017-11-16 02:00:00+00:00    v1    v2    2
             2017-11-16 03:00:00+00:00  None  None    3
             2017-11-16 03:00:00+00:00    v1    v2    3
             2017-11-16 04:00:00+00:00  None  None    4
             2017-11-16 04:00:00+00:00    v1    v2    4
             2017-11-16 05:00:00+00:00  None  None    5
             2017-11-16 05:00:00+00:00    v1    v2    5
             2017-11-16 06:00:00+00:00  None  None    6
             2017-11-16 06:00:00+00:00    v1    v2    6
             2017-11-16 07:00:00+00:00  None  None    7
             2017-11-16 07:00:00+00:00    v1    v2    7
             2017-11-16 08:00:00+

In [ ]:

print("Delete database: " + dbname)
client.drop_database(dbname)

In [1]:
import numpy as np
import pandas as pd
import time
import requests
url = 'http://localhost:32768/write'
params = {"db": "base", "u": "root", "p": "root"}

In [2]:
def read_data():
    with open('data/gas_ft.csv') as f:
        return [x.split(',') for x in f.readlines()[1:]]

a = read_data()

In [3]:
a[0]

['67', 'm\xc2\xb3', '0.34', '2014-07-10T00:30:00.000Z\n']

In [22]:
#payload = "elec,id=500 value=24 2018-03-05T19:31:00.000Z\n"
payload = "elec,id=500 value=24 "#+str(pd.to_datetime('2018-03-05T19:29:00.000Z\n').value // 10 ** 9)
r = requests.post(url, params=params, data=payload)

'1520278260'

In [ ]:
# -*- coding: utf-8 -*-
"""Tutorial how to use the class helper `SeriesHelper`."""

from influxdb import InfluxDBClient
from influxdb import SeriesHelper

# InfluxDB connections settings
host = 'localhost'
port = 8086
user = 'root'
password = 'root'
dbname = 'mydb'

myclient = InfluxDBClient(host, port, user, password, dbname)

# Uncomment the following code if the database is not yet created
# myclient.create_database(dbname)
# myclient.create_retention_policy('awesome_policy', '3d', 3, default=True)


class MySeriesHelper(SeriesHelper):
    """Instantiate SeriesHelper to write points to the backend."""

    class Meta:
        """Meta class stores time series helper configuration."""

        # The client should be an instance of InfluxDBClient.
        client = myclient

        # The series name must be a string. Add dependent fields/tags
        # in curly brackets.
        series_name = 'events.stats.{server_name}'

        # Defines all the fields in this time series.
        fields = ['some_stat', 'other_stat']

        # Defines all the tags for the series.
        tags = ['server_name']

        # Defines the number of data points to store prior to writing
        # on the wire.
        bulk_size = 5

        # autocommit must be set to True when using bulk_size
        autocommit = True


# The following will create *five* (immutable) data points.
# Since bulk_size is set to 5, upon the fifth construction call, *all* data
# points will be written on the wire via MySeriesHelper.Meta.client.
MySeriesHelper(server_name='us.east-1', some_stat=159, other_stat=10)
MySeriesHelper(server_name='us.east-1', some_stat=158, other_stat=20)
MySeriesHelper(server_name='us.east-1', some_stat=157, other_stat=30)
MySeriesHelper(server_name='us.east-1', some_stat=156, other_stat=40)
MySeriesHelper(server_name='us.east-1', some_stat=155, other_stat=50)

# To manually submit data points which are not yet written, call commit:
MySeriesHelper.commit()

# To inspect the JSON which will be written, call _json_body_():
MySeriesHelper._json_body_()

In [9]:
for metric in a[:]:
    payload = "elec,id="+str(metric[0])+" value="+str(metric[2])+" "+str(pd.to_datetime(metric[3]).value // 10 ** 9)+"\n"
    #payload = "water,id="+str(metric[0])+" value="+str(metric[2])+"\n"
    r = requests.post(url, params=params, data=payload)

KeyboardInterrupt: 

In [9]:
def read_data():
    with open('data/water_ft.csv') as f:
        return [x.split(',') for x in f.readlines()[1:]]

a = read_data()

In [10]:
a[0]

['68', 'm\xc2\xb3', '1.18', '2014-07-10T00:30:00.000Z\n']

In [ ]:
for metric in a[1000:3000]:
    #payload = "gas,id="+str(metric[0])+" value="+str(metric[2])+" "+str(pd.to_datetime(metric[3]).value // 10 ** 9)+"\n"
    payload = "water,id="+str(metric[0])+" value="+str(metric[2])+"\n"
    r = requests.post(url, params=params, data=payload)
    time.sleep(1)